# Create a Graph from a text

Este caderno demonstra como extrair grafo de qualquer texto usando o criador de grafos

Passos:
- Definir uma ontologia
- Carregar uma lista de exemplos de blocos de texto. Usaremos o resumo do Senhor dos Anéis desta página da Wikipedia.
- Criar gráfico usando um modelo de código aberto usando APIs Groq.
- Salve o gráfico no banco de dados Neo4j
- Visualizar



Carregar as funções do criador de grafos ->

In [1]:
## no Linux no terminal rodar:
# export GROQ_API_KEY='secret'

## no Windows no Powershell rodar:
# $env:GROQ_API_KEY = "secret"

# %pip install --upgrade httpx
# %pip install --upgrade groq
# %pip install python-certifi-win32
# %pip install --upgrade certifi

import certifi
print(certifi.where())

c:\Users\marco\anaconda3\Lib\site-packages\certifi\cacert.pem


In [2]:
%pip show httpx

Name: httpx
Version: 0.27.0
Summary: The next generation HTTP client.
Home-page: 
Author: 
Author-email: Tom Christie <tom@tomchristie.com>
License: 
Location: c:\Users\marco\anaconda3\Lib\site-packages
Requires: anyio, certifi, httpcore, idna, sniffio
Required-by: fastapi, googletrans, gradio, gradio_client, groq, qdrant-client
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip show httpx

Name: httpx
Version: 0.13.3
Summary: The next generation HTTP client.
Home-page: https://github.com/encode/httpx
Author: Tom Christie
Author-email: tom@tomchristie.com
License: BSD
Location: c:\Users\marco\anaconda3\Lib\site-packages
Requires: certifi, chardet, hstspreload, httpcore, idna, rfc3986, sniffio
Required-by: fastapi, googletrans, gradio, gradio_client, groq, qdrant-client
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
import platform
import httpx
import ssl
from groq import Groq

if platform.system() == 'Linux':
    folder = '../'
elif platform.system() == 'Windows':
    try:
        folder = '../'
        pathfilename = os.path.join(folder, 'secrets.json')
        with open(pathfilename) as file:
            data = json.load(file)
    except FileNotFoundError:
        folder = '../../../'
        pathfilename = os.path.join(folder, 'secrets.json')
        with open(pathfilename) as file:
            data = json.load(file)
else:
    raise SystemError('Sistema operacional não suportado')

try:
    httpx_client = httpx.Client(
        verify=False,  
        limits=httpx.Limits(max_keepalive_connections=None, max_connections=None),
        transport=httpx.HTTPTransport(retries=3)
    )

    client = Groq(api_key=data.get('groq'), http_client=httpx_client)

    if not client.api_key:  # Verifica se a chave de API foi obtida corretamente
        raise ValueError("Chave de API Groq não encontrada ou inválida.")

    immunology_questions = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "What are the main scientific questions of interest in the area of immunology and immunopathology? Escreva a resposta em português do Brasil",
            }
        ],
        model="llama3-70b-8192",
    )

    print(immunology_questions.choices[0].message.content)

except (httpx.ConnectError, httpx.ReadTimeout) as e:
    if isinstance(e.reason, ssl.SSLError):
        print(f"Erro de certificado SSL: {e.reason} ({e})")  # Mensagem de erro mais detalhada
    else:
        print(f"Erro de conexão: {e}")
except ValueError as e:  # Captura erro de chave de API inválida
    print(e)

As principais questões científicas de interesse na área de imunologia e imunopatologia incluem:

1. **Como o sistema imune distingue entre o eu e o não-eu?**: Este é um desafio fundamental na imunologia, pois o sistema imune precisa distinguir entre as células do próprio corpo e as células estranhas, como bactérias e vírus, para evitar respostas imunes autoimunes.

2. **Como a imunidade celular e humoral se coordinam para controlar infecções?**: A resposta imune é mediada por células imunes, como linfócitos T e B, que trabalham juntas para eliminar patógenos. Entender como essas células se comunicam e se coordenam é fundamental para desenvolver estratégias de prevenção e tratamento de doenças.

3. **Quais são os mecanismos moleculares que regulam a resposta imune?**: A sinalização molecular é crucial para a ativação e regulação da resposta imune. Entender como as moléculas sinalizadoras, como citocinas e quimiocinas, regulam a resposta imune pode levar a terapias mais eficazes.

4. **C

In [3]:
# import os, json, platform
# import httpx, ssl
# from groq import Groq

# if platform.system() == 'Linux':
#     folder = '../'
# elif platform.system() =='Windows':
#     try:
#         folder = '../'
#         pathfilename = os.path.join(folder,'secrets.json')
#         with open(pathfilename) as file:
#             # Carregar seu conteúdo e torná-lo um novo dicionário
#             data = json.load(file)        
#     except:
#         folder = '../../../'
#         pathfilename = os.path.join(folder,'secrets.json')
#         with open(pathfilename) as file:
#             # Carregar seu conteúdo e torná-lo um novo dicionário
#             data = json.load(file)        
# else:
#     print('Sistema operacional não suportado')

# try:
#     # Criar um cliente HTTPx com verificação SSL desativada e limites customizados
#     httpx_client = httpx.Client(
#         verify=False,
#         limits=httpx.Limits(max_keepalive_connections=None, max_connections=None),
#         transport=httpx.HTTPTransport(retries=3)
#     )

#     client = Groq(
#         # api_key=os.environ.get("GROQ_API_KEY"),
#         api_key=data.get('groq'),
#         http_client=httpx_client
#     )


#     immunology_questions = client.chat.completions.create(
#         messages=[
#             {
#                 "role": "user",
#                 "content": "What are the main scientific questions of interest in the area of immunology and immunopathology? Escreva a resposta em português do Brasil",
#             }
#         ],
#         # model="llama3-8b-8192",
#         model = "llama3-70b-8192",
#     )

#     print(immunology_questions.choices[0].message.content)

# except httpx.ConnectError as e:
#     if isinstance(e.reason, ssl.SSLError):
#         print("Erro de certificado SSL:", e.reason)
#         # ... (investigar o certificado SSL da Groq e corrigi-lo) ...
#     else:
#         print("Erro de conexão:", e)

In [4]:
immunology_platform_communication = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What are the main scientific platforms for carrying out research in immunology and immunopathology? Escreva a resposta em português do Brasil",
        }
    ],
    model="llama3-8b-8192",
)

print(immunology_platform_communication.choices[0].message.content)

Há várias plataformas científicas importantes para realização de pesquisas em imunologia e imunopatologia. Aqui estão algumas das principais:

1. Nature Immunology: É uma revista de alto impacto que publica artigos originais sobre imunologia humana e animal.
2. Journal of Experimental Medicine: É uma revista líder em imunologia e biologia celular, que publica artigos originais sobre imunopatologia, imunobiologia e biologia do câncer.
3. Immunity: É uma revista que concentra-se em imunologia clínica, experimentação e temas relacionados à patalogia imune.
4. Blood: É uma revista que publica artigos sobre hematologia, Hemostasia e imunologia sanguínea.
5. European Journal of Immunology: É uma revista que publica artigos originais sobre imunologia humana e animal, incluindo imunologia clínica e experimental.
6. Journal of Immunology: É uma revista que publica artigos originais sobre imunologia humana e animal, incluindo imunopatologia e imunobiologia.
7. Journal of Allergy and Clinical Imm

In [5]:
immunology_tech_platforms = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What are the main scientific technology platforms for carrying out scientific research questions in immunology and immunopathology? Escreva a resposta em português do Brasil",
        }
    ],
    model="llama3-8b-8192",
)

print(immunology_tech_platforms.choices[0].message.content)

Excelente pergunta!

As principais plataformas científicas e tecnológicas utilizadas para realizar pesquisas científicas em imunologia e imunopatologia são:

1. Microscopia de Fluorescência - Utilizada para visualizar e analisar a expressão de moléculas de sinalização e de adesão nas células do sistema imunológico.
2. Cytometry by Fluorescence - Uso de feitores de fluorescência para medir a expressão de moléculas de sinalização e de adesão nas células do sistema imunológico.
3. Flow Cytometry - Utilizado para analisar a expressão de moléculas de sinalização e de adesão nas células do sistema imunológico.
4. ELISA (Enzyme-Linked Immunosorbent Assay) - Utilizado para medir a concentração de anticorpos e outras proteínas presentes no sangue e nos tecidos.
5. PCR (Polymerase Chain Reaction) - Uma técnica de amplificação de DNA utilizada para detectar e quantificar genes associados a doenças imunológicas.
6. qPCR (Quantitative PCR) - Uma variante da PCR que fornece informações quantitativas

In [6]:
immunology_ontologies = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What are the main ontologies to encompass the semantics concepts in scientific technology platforms in immunology and immunopathology? Escreva a resposta em português do Brasil",
        }
    ],
    model="llama3-8b-8192",
)

print(immunology_ontologies.choices[0].message.content)

Um desafio interessante!

As principais ontologias que abordam conceitos semânticos em plataformas de tecnologia científica relacionadas à imunologia e patologia imunológica são:

1. **Biologie de Immunologie** (Immune System Ontology): desenvolvida pelo Universidade de Oxford, essa ontologia descreve conceitos e relações importantes na imunologia, como células imunes, citocinas, antígenos e respostas imunológicas.
2. **MGI Mouse Gene Functional Ontology** (MGI): embora não exclusivamente centrada em imunologia, a MGI é uma ontologia ampla que aborda genes, proteínas e fenótipos, incluindo conceitos relacionados à imunologia e patologia.
3. **The Gene Ontology (GO)**: enquanto não especializada em imunologia, a GO é amplamente utilizada em muitos campos da biologia e saúde, incluindo a imunologia, para descrever e relacionar genes e seu comportamento.
4. **The Immunological Database and Analysis Portal (ImmPort)**: embora não uma ontologia propriamente dita, o ImmPort é uma ferramenta 

In [7]:
# %pip install jupyter_contrib_nbextensions
# %jupyter contrib nbextension install --user
# %jupyter nbextension enable --py --sys-prefix widgetsnbextension
# %jupyter nbextension enable mermaid --py --sys-prefix

# Síntese da arquitetura

    Integração de ontologias em Grafo de Conhecimento

```mermaid
graph LR
    subgraph Camada de Domínio
        Ontology[Ontology]
        Article[Article]
        Route[Route]
    end

    subgraph Camada de Aplicação
        NLP[NLP]
        GraphKnowledge[Graph Knowledge]
        MachineLearning[Machine Learning]
        Recommendation[Recommendation]
    end

    subgraph Camada de Infraestrutura
        Persistence[Persistence]
        UI[UI]
    end

    Ontology --> GraphKnowledge
    Article --> NLP
    Article --> GraphKnowledge
    NLP --> GraphKnowledge
    GraphKnowledge --> MachineLearning
    MachineLearning --> Recommendation
    Recommendation --> Route
    Persistence --> GraphKnowledge
    Persistence --> MachineLearning
    UI --> NLP
    Route --> UI    

```mermaid
classDiagram
    class Ontology {
        -ontologies: List<OntologyConcept>
        +addOntologyConcept(concept: OntologyConcept): void
        +getOntologyConcepts(): List<OntologyConcept>
        +getOntologyConceptByName(name: string): OntologyConcept
    }

    class OntologyConcept {
        -name: string
        -description: string
        -relationships: List<OntologyRelationship>
        +getName(): string
        +getDescription(): string
        +getRelationships(): List<OntologyRelationship>
    }

    class OntologyRelationship {
        -source: OntologyConcept
        -target: OntologyConcept
        -type: string
        +getSource(): OntologyConcept
        +getTarget(): OntologyConcept
        +getType(): string
    }


```mermaid
classDiagram
    class Article {
        -title: string
        -abstract: string
        -authors: List<string>
        -publicationDate: Date
        -entities: List<Entity>
        -relationships: List<Relationship>
        +getTitle(): string
        +getAbstract(): string
        +getAuthors(): List<string>
        +getPublicationDate(): Date
        +getEntities(): List<Entity>
        +getRelationships(): List<Relationship>
    }

    class Entity {
        -name: string
        -type: string
        +getName(): string
        +getType(): string
    }

    class Relationship {
        -source: Entity
        -target: Entity
        -type: string
        +getSource(): Entity
        +getTarget(): Entity
        +getType(): string
    }


```mermaid
classDiagram
    class Route {
        -steps: List<Step>
        -technologies: List<Technology>
        -applications: List<string>
        +getSteps(): List<Step>
        +getTechnologies(): List<Technology>
        +getApplications(): List<string>
    }

    class Step {
        -description: string
        -entities: List<Entity>
        +getDescription(): string
        +getEntities(): List<Entity>
    }

    class Technology {
        -name: string
        -description: string
        +getName(): string
        +getDescription(): string
    }

In [1]:
import os
filepath = os.path.join('source','adapters','input','jupyter_notebooks','diagrams')
os.chmod(filepath, 0o755)  # Set permissions to rwxr-xr-x
os.listdir(filepath)

['01_nosCNPq.png',
 'aplications_cml_diagram',
 'aplications_cml_diagram.png',
 'aplications_gml_diagram',
 'aplications_gml_diagram.png',
 'azure_architecture_diagram.png',
 'bpmn',
 'cadeia_valor_processos_n00.png',
 'cadeia_valor_processos_n01.png',
 'detailed_ml_pipeline_architecture_diagram.png',
 'diagram5x5',
 'diagram5x5.png',
 'diagram_procurement.gv',
 'diagram_procurement.gv.pdf',
 'gml_architecture.png',
 'gml_simplified_architecture_diagram.png',
 'gml_simplified_architecture_diagram_azure.png',
 'PRISMA_01.gv',
 'PRISMA_01.pdf',
 'prisma_flowchart.png',
 'prisma_simple_flowchart.png',
 'recomendar_rotas_tecnológicas.png',
 'simple_prisma_diagram.png',
 'simple_prisma_flowchart.png']

In [2]:
from diagrams import Cluster, Diagram
from diagrams.custom import Custom
from diagrams.oci.compute import OKE
from diagrams.oci.storage import DataTransfer
from diagrams.oci.governance import Compartments, Audit
from diagrams.oci.connectivity import CustomerDatacenter, NATGateway
from diagrams.oci.database import BigdataService, DatabaseService, Dcat
from diagrams.oci.monitoring import HealthCheck, Queue, Workflow, Search
from diagrams.onprem.inmemory import Redis
from diagrams.generic.os import Windows
from diagrams.generic.storage import Storage
from diagrams.generic.compute import Rack
from diagrams.generic.place import Datacenter
from diagrams.generic.device import Mobile, Tablet
from diagrams.aws.general import User, Toolkit
from diagrams.aws.compute import ECS, EKS, Lambda
from diagrams.aws.cost import CostExplorer
from diagrams.aws.ml import SagemakerModel, DeepLearningAmis
from diagrams.aws.storage import ElasticFileSystemEFSFileSystem
from diagrams.aws.database import RedshiftDenseComputeNode as KnowledgeGraph, DynamodbAttributes as Ontology
from diagrams.programming.language import Python, NodeJS
from diagrams.programming.framework import React, Flask, GraphQL
from diagrams.programming.flowchart import MultipleDocuments
# https://diagrams.mingrammer.com/docs/nodes/programming

filepath = os.path.join('source','adapters','input','jupyter_notebooks','diagrams')
with Diagram("Recomendar Rotas Tecnológicas", show=False, direction="TB", node_attr={"shape": "plaintext"}, filename=filepath) as diag:
    with Cluster("CAMADA DE DOMÍNIO"):
        article = Dcat("Article")  
        entity = Storage("Entity")
        
        with Cluster("Módulo Semântico"):
            ontology = Search("Ontology")
            concept = BigdataService("Concept")
            relationship = DatabaseService("Relationship")

        with Cluster("Módulo Rota Tecnológica"):
            route = Workflow("Route")
            step = Queue("Step")
            technology = Toolkit("Technology")
        with Cluster("Módulo CEIS"):
            ceis = Datacenter("CEIS")
            competence = CustomerDatacenter("Competence")
            objective = HealthCheck("Objective")
            platform = OKE("Platform")
            product = Compartments("Product")

    with Cluster("CAMADA DE APLICAÇÃO"):
        nlp = Python("NLP")
        with Cluster("Módulo GML"):
            graph_knowledge = DeepLearningAmis("Graph Knowledge", width="1.5") 
            machine_learning = SagemakerModel("Machine Learning", width="1.5")
        recommendation = Audit("Recommendation")
        integration = CostExplorer("Integration")

    with Cluster("CAMADA DE INFRAESTRUTURA"):
        persistence = Python("Persistence")
        ui = NodeJS("UI")

    user = User("Usuários")

    ontology >> graph_knowledge
    ontology >> concept
    ontology >> relationship
    concept >> graph_knowledge
    relationship >> graph_knowledge
    article >> nlp
    article >> entity
    article >> graph_knowledge
    entity >> graph_knowledge
    nlp >> graph_knowledge
    nlp >> entity
    nlp >> relationship
    graph_knowledge >> machine_learning
    machine_learning >> recommendation
    machine_learning >> competence
    machine_learning >> objective
    machine_learning >> platform
    machine_learning >> product
    recommendation >> ui
    recommendation >> machine_learning
    integration >> graph_knowledge
    integration >> ontology
    integration >> article
    integration >> ceis
    persistence >> graph_knowledge
    persistence >> machine_learning
    ui >> nlp
    ceis >> competence
    ceis >> objective
    ceis >> platform
    ceis >> product
    competence >> recommendation
    objective >> recommendation
    platform >> recommendation
    product >> recommendation

    route >> step
    route >> technology
    step >> entity
    technology >> graph_knowledge

    ui >> user
    user >> ui

diag

PermissionError: [Errno 13] Permission denied: 'source\\adapters\\input\\jupyter_notebooks\\diagrams'

In [11]:
# %pip show diagrams
# %pip install --upgrade diagrams

## Detalhamento da interação entre módulos

```mermaid
graph LR
    subgraph Camada de Domínio
        Ontology[Ontology]
        Concept[Concept]
        Relationship[Relationship]
        Article[Article]
        Entity[Entity]
        Route[Route]
        Step[Step]
        Technology[Technology]
        CEIS[CEIS]
        Competence[Competence]
        Objective[Objective]
        Platform[Platform]
        Product[Product]
    end

    subgraph Camada de Aplicação
        NLP[NLP]
        GraphKnowledge[Graph Knowledge]
        MachineLearning[Machine Learning]
        Recommendation[Recommendation]
        Integration[Integration]
    end

    subgraph Camada de Infraestrutura
        Persistence[Persistence]
        UI[UI]
    end

    Ontology --> GraphKnowledge
    Ontology --> Concept
    Ontology --> Relationship
    Concept --> GraphKnowledge
    Relationship --> GraphKnowledge
    Article --> NLP
    Article --> Entity
    Article --> GraphKnowledge
    Entity --> GraphKnowledge
    NLP --> GraphKnowledge
    NLP --> Entity
    NLP --> Relationship
    GraphKnowledge --> MachineLearning
    MachineLearning --> Recommendation
    MachineLearning --> Competence
    MachineLearning --> Objective
    MachineLearning --> Platform
    MachineLearning --> Product
    Recommendation --> UI
    Integration --> GraphKnowledge
    Integration --> Ontology
    Integration --> Article
    Integration --> CEIS
    Persistence --> GraphKnowledge
    Persistence --> MachineLearning
    UI --> NLP
    CEIS --> Competence
    CEIS --> Objective
    CEIS --> Platform
    CEIS --> Product
    Competence --> Recommendation
    Objective --> Recommendation
    Platform --> Recommendation
    Product --> Recommendation

    Route --> Step
    Route --> Technology
    Step --> Entity
    Technology --> GraphKnowledge 


```mermaid
classDiagram
    Ontology <|-- Concept
    Ontology <|-- Relationship

    class Ontology {
        -ontologies: List<OntologyConcept>
        +addOntologyConcept(concept: OntologyConcept): void
        +getOntologyConcepts(): List<OntologyConcept>
        +getOntologyConceptByName(name: string): OntologyConcept
        +getRelationships(): List<OntologyRelationship>
        +getRelationshipsByType(type: string): List<OntologyRelationship>
        +findRelatedConcepts(concept: OntologyConcept): List<OntologyConcept>
    }

    class OntologyConcept {
        -id: string
        -name: string
        -description: string
        -synonyms: List<string>
        -relationships: List<OntologyRelationship>
        +getId(): string
        +getName(): string
        +getDescription(): string
        +getSynonyms(): List<string>
        +getRelationships(): List<OntologyRelationship>
        +getRelationshipsByType(type: string): List<OntologyRelationship>
    }

    class OntologyRelationship {
        -id: string
        -source: OntologyConcept
        -target: OntologyConcept
        -type: string
        +getId(): string
        +getSource(): OntologyConcept
        +getTarget(): OntologyConcept
        +getType(): string
    }


```mermaid
classDiagram
    Article <|-- Entity

    class Article {
        -id: string
        -title: string
        -abstract: string
        -authors: List<string>
        -publicationDate: Date
        -entities: List<Entity>
        -relationships: List<Relationship>
        +getId(): string
        +getTitle(): string
        +getAbstract(): string
        +getAuthors(): List<string>
        +getPublicationDate(): Date
        +getEntities(): List<Entity>
        +getRelationships(): List<Relationship>
        +addEntity(entity: Entity): void
        +addRelationship(relationship: Relationship): void
    }

    class Entity {
        -id: string
        -name: string
        -type: string
        -ontologyConcept: OntologyConcept
        +getId(): string
        +getName(): string
        +getType(): string
        +getOntologyConcept(): OntologyConcept
    }

    class Relationship {
        -id: string
        -source: Entity
        -target: Entity
        -type: string
        -ontologyRelationship: OntologyRelationship
        +getId(): string
        +getSource(): Entity
        +getTarget(): Entity
        +getType(): string
        +getOntologyRelationship(): OntologyRelationship
    }


```mermaid
classDiagram

    CEIS <|-- Competence
    CEIS <|-- Objective
    CEIS <|-- Platform
    CEIS <|-- Product

    class CEIS {
        -competences: List<Competence>
        -objectives: List<Objective>
        -platforms: List<Platform>
        -products: List<Product>
        +addCompetence(competence: Competence): void
        +getCompetences(): List<Competence>
        +addObjective(objective: Objective): void
        +getObjectives(): List<Objective>
        +addPlatform(platform: Platform): void
        +getPlatforms(): List<Platform>
        +addProduct(product: Product): void
        +getProducts(): List<Product>
    }

    class Competence {
        -id: string
        -name: string
        -description: string
        +getId(): string
        +getName(): string
        +getDescription(): string
    }

    class Objective {
        -id: string
        -name: string
        -description: string
        +getId(): string
        +getName(): string
        +getDescription(): string
    }

    class Platform {
        -id: string
        -name: string
        -description: string
        +getId(): string
        +getName(): string
        +getDescription(): string
    }

    class Product {
        -id: string
        -name: string
        -description: string
        +getId(): string
        +getName(): string
        +getDescription(): string
    }


```mermaid
classDiagram
    Route <|-- Step
    Route <|-- Technology

    class Route {
        -id: string
        -steps: List<Step>
        -technologies: List<Technology>
        -applications: List<string>
        +getId(): string
        +getSteps(): List<Step>
        +getTechnologies(): List<Technology>
        +getApplications(): List<string>
        +addStep(step: Step): void
        +addTechnology(technology: Technology): void
    }

    class Step {
        -id: string
        -description: string
        -entities: List<Entity>
        +getId(): string
        +getDescription(): string
        +getEntities(): List<Entity>
        +addEntity(entity: Entity): void
    }

    class Technology {
        -id: string
        -name: string
        -description: string
        -ontologyConcept: OntologyConcept
        +getId(): string
        +getName(): string
        +getDescription(): string
        +getOntologyConcept(): OntologyConcept
    }


## Explicação

O diagrama apresenta a arquitetura proposta, onde algumas relações podem ser destacadas para maior clareza e precisão:

### Módulo de Integração:
A seta de Integration para Graph Knowledge indica que a integração alimenta o grafo com dados externos, bem como conecta a Ontology, Article, e CEIS, já que extrai informações desses domínios para integrá-las ao grafo.
NLP (Processamento de Linguagem Natural). Já as setas de Article e UI para NLP indicam que o NLP processa dados de artigos e interage com a interface do usuário, e NLP também, se conecta a Entity e Relationship, por extrair essas informações dos artigos.

### Módulo de Graph Knowledge:
As setas de Ontology, Concept, Relationship, Article, Entity, Technology e Integration para Graph Knowledge estão corretas, mostrando que o grafo armazena informações de todas essas fontes.

### Módulo de Machine Learning:
As setas de Graph Knowledge e Persistence para Machine Learning indicam que o aprendizado de máquina utiliza dados do grafo e do armazenamento persistente, e se conectar a Competence, Objective, Platform e Product do domínio CEIS, pois o aprendizado de máquina usa essas informações para modelar as recomendações.

### Módulo de Recommendation:
As setas de Machine Learning, Competence, Objective, Platform e Product para Recommendation estão corretas, mostrando que a recomendação é baseada em aprendizado de máquina e nas informações do CEIS.

### UI (Interface do Usuário): 
A seta de Recommendation para UI está correta, indicando que a interface exibe as recomendações.
A seta de UI para NLP também está correta, representando a interação do usuário com o sistema através de linguagem natural.

### Persistência: 
A seta de Persistence para Graph Knowledge está correta, mas também poderia haver uma seta para Machine Learning, já que alguns modelos podem precisar armazenar dados persistentes (como checkpoints de treinamento).

### Domínio CEIS: 
As setas dentro do domínio CEIS estão corretas, representando as relações entre competências, objetivos, plataformas e produtos.

### Obs.: 
Bidirecionalidade: Algumas setas poderiam ser bidirecionais, como entre Graph Knowledge e Machine Learning, para indicar que o aprendizado de máquina não apenas utiliza dados do grafo, mas também pode atualizar o grafo com novos conhecimentos.

In [1]:
from knowledge_graph_maker import GraphMaker, Ontology, GroqClient, OpenAIClient
from knowledge_graph_maker import Document

In [7]:
immunology_graph_ontologies = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "How integrate in a knowledge graph the main REAL ontologies to encompass the semantics concepts in scientific technology platforms for carrying out scientific research questions in immunology and immunopathology? Escreva a resposta em português do Brasil",
        }
    ],
    model="llama3-8b-8192",
)

print(immunology_graph_ontologies.choices[0].message.content)

Integrating main REAL ontologies in a knowledge graph to encompass the semantics concepts in scientific technology platforms for carrying out scientific research questions in immunology and immunopathology is a complex task that requires a clear understanding of the real-world domain, the ontologies, and the knowledge graph architecture. Here's a suggested approach:

**Choose the relevant ontologies:**

1. **SNOMED-CT** (Systematized Nomenclature of Medicine – Clinical Terms): a comprehensive ontology for biomedical concepts, including immunology and immunopathology.
2. **MeSH** (Medical Subject Headings): a thesaurus for medical topics, including immunology and immunopathology.
3. **NCI Thesaurus** (National Cancer Institute Thesaurus): a controlled vocabulary for cancer research, including immunology and immunopathology.
4. **HO** (Human Ontology): a comprehensive ontology for human biology, including immunology and immunopathology.

**Integrate the ontologies:**

1. **Merge the onto

In [8]:
immunology_graph_ontologies = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "How integrate in a knowledge graph the main REAL ontologies to encompass the semantics concepts in scientific technology platforms for carrying out scientific research questions in immunology and immunopathology? Escreva a resposta em português do Brasil",
        }
    ],
    model="llama3-70b-8192",
)

print(immunology_graph_ontologies.choices[0].message.content)

Para integrar as principais ontologias reais em um grafo de conhecimento para abranger os conceitos semânticos em plataformas de tecnologia científica para realizar perguntas de pesquisa científica em imunologia e imunopatologia, é necessário seguir os seguintes passos:

1. **Identificar as ontologias relevantes**: É importante identificar as principais ontologias reais em imunologia e imunopatologia, como:
	* Ontologia de Doenças (Disease Ontology, DOID)
	* Ontologia de Genômica e Proteômica (Gene Ontology, GO)
	* Ontologia de Anatomia (Anatomy Ontology, Uberon)
	* Ontologia de Patologia (Pathology Ontology, PATO)
	* Ontologia de Imunologia (Immunology Ontology, IEDB)
2. **Selecionar as ontologias mais relevantes**: Selecionar as ontologias mais relevantes para o domínio de imunologia e imunopatologia, considerando a sua abrangência, precisão e importância.
3. **Criar um grafo de conhecimento**: Criar um grafo de conhecimento que integre as ontologias selecionadas, utilizando tecnolog

# Definir a ontologia.

A ontologia é um modelo formatado com o seguinte esquema.

```python
class Ontology(BaseModel):
    label: List[Union[str, Dict]]
    relationships: List[str]
```

Por exemplo, vamos usar resumos dos livros LOTR da página da Wikipedia. Copiei-o para um arquivo para facilitar a importação

In [6]:
# from lotr_wikipedia_summary import lord_of_the_rings_wikipedia_summary as example_text_list
# len(example_text_list)

Aqui está a ontologia que usaremos para os resumos do LOTR ->

In [8]:
BaseModel

'llama3-70b-8192'

In [10]:
from typing import List, Union, Dict
BaseModel='llama3-70b-8192'
class Ontology(BaseModel):
    label: List[Union[str, Dict]]
    relationships: List[str]

ontology = Ontology(
    labels=[
        {"Person": "Person name without any adjectives, Remember a person may be referenced by their name or using a pronoun"},
        {"Object": "Do not add the definite article 'the' in the object name"},
        {"Event": "Event event involving multiple people. Do not include qualifiers or verbs like gives, leaves, works etc."},
        "Place",
        "Document",
        "Organisation",
        "Action",
        {"Miscellaneous": "Any important concept can not be categorised with any other given label"},
    ],
    relationships=[
        "Relation between any pair of Entities"
        ],
)

TypeError: str() argument 2 must be str, not tuple

## Select a Model

Groq support the following models at present. 

*LLaMA3 8b*
Model ID: llama3-8b-8192

*LLaMA3 70b*
Model ID: llama3-70b-8192

*Mixtral 8x7b*
Model ID: mixtral-8x7b-32768

*Gemma 7b*
Model ID: gemma-7b-it


Selecting a model for this example ->


In [11]:

## Groq models
# model = "mixtral-8x7b-32768"
# model ="llama3-8b-8192"
model = "llama3-70b-8192"
# model="gemma-7b-it"

## Open AI models
oai_model="gpt-3.5-turbo"

## Use Groq
# llm = GroqClient(model=model, temperature=0.1, top_p=0.5)

## OR Use OpenAI
model = "llama3-70b-8192"
llm = OpenAIClient(model=oai_model, temperature=0.1, top_p=0.5)


NameError: name 'OpenAIClient' is not defined

## Criar documentos a partir de blocos de texto.
Documents é um modelo com o seguinte esquema

```python
class Document(BaseModel):
    text: str
    metadata: dict
```

Os metadados que adicionamos ao documento aqui são copiados para cada relação extraída do documento. Na maioria das vezes, os pares de nós têm múltiplas relações entre si. Os metadados ajudam a adicionar mais contexto a essas relações

Neste exemplo, estou gerando um resumo do bloco de texto e o carimbo de data/hora da execução para ser usado como metadados.

In [20]:
import datetime
current_time = str(datetime.datetime.now())

class Document(BaseModel):
    text: str
    metadata: dict

graph_maker = GraphMaker(ontology=ontology, llm_client=llm, verbose=False)

def generate_summary(text):
    SYS_PROMPT = (
        "Succintly summarise the text provided by the user. "
        "Respond only with the summary and no other comments"
    )
    try:
        summary = llm.generate(user_message=text, system_message=SYS_PROMPT)
    except:
        summary = ""
    finally:
        return summary

docs = map(
    lambda t: Document(text=t, metadata={"summary": generate_summary(t), 'generated_at': current_time}),
    example_text_list
)

TypeError: str() argument 'encoding' must be str, not tuple

In [21]:
def generate_metadata(text):
    current_time = str(datetime.datetime.now())
    summary = generate_summary(text)
    return {"summary": summary, "generated_at": current_time}

docs = map(
    lambda t: Document(text=t, metadata=generate_metadata(t)),
    example_text_list,
)

NameError: name 'example_text_list' is not defined

## Create Graph
Finally run the Graph Maker to generate graph. 

In [ ]:
graph = graph_maker.from_documents(
    list(docs), 
    delay_s_between=0 ## delay_s_between because otherwise groq api maxes out pretty fast. 
    ) 
print("Total number of Edges", len(graph))

In [ ]:
for edge in graph:
    print(edge.model_dump(exclude=['metadata']), "\n\n")

# Save the Graph to Neo4j 

In [10]:
from knowledge_graph_maker import Neo4jGraphModel

create_indices = False
neo4j_graph = Neo4jGraphModel(edges=graph, create_indices=create_indices)

neo4j_graph.save()

aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
a

149